Importar librerías y cargar variables de entorno

In [55]:
# Importar librerías necesarias
import pandas as pd
import pyodbc
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Recuperar variables de entorno
server = os.getenv('SQL_SERVER')
username = os.getenv('SQL_USERNAME')
password = os.getenv('SQL_PASSWORD')

# Construir la cadena de conexión
cnxn_str = f"Driver={{ODBC Driver 18 for SQL Server}};Server={server},1433;Database=employeedirectorydb;UID={username};Pwd={password};Encrypt=yes;TrustServerCertificate=no;Connection Timeout=220;"

# Intentar conectar
try:
    conn = pyodbc.connect(cnxn_str)
    print("Conexión exitosa.")
    conn.close()
except Exception as e:
    print(f"Error al conectar a SQL Server: {e}")


Conexión exitosa.


Extraer datos desde el archivo CSV

In [56]:
# Definir una función para extraer datos
def extract_data(csv_file_path, delimiter=','):
    """
    Extrae datos desde un archivo CSV.

    :param csv_file_path: Ruta del archivo CSV.
    :param delimiter: Separador del archivo CSV. Por defecto es una coma.
    :return: Un DataFrame con los datos extraídos o None si ocurre un error.
    """
    try:
        # Intentar leer con UTF-8
        data = pd.read_csv(csv_file_path, encoding='utf-8', delimiter=delimiter)
    except UnicodeDecodeError:
        try:
            # Intentar leer con Latin-1 si UTF-8 falla
            data = pd.read_csv(csv_file_path, encoding='ISO-8859-1', delimiter=delimiter)
        except Exception as e:
            # Capturar cualquier otro error
            print(f'Error al leer el archivo {csv_file_path} con ISO-8859-1:', e)
            return None
    except Exception as e:
        # Capturar errores generales
        print(f'Error al leer el archivo {csv_file_path} con UTF-8:', e)
        return None
    return data

# Extraer datos
csv_file_path = 'netflix_titles.csv' 
data = extract_data(csv_file_path, delimiter=',') 

# Verificar la estructura de los datos
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8809 entries, 0 to 8808
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       8809 non-null   object 
 1   type          8809 non-null   object 
 2   title         8809 non-null   object 
 3   director      6175 non-null   object 
 4   cast          7984 non-null   object 
 5   country       7978 non-null   object 
 6   date_added    8799 non-null   object 
 7   release_year  8809 non-null   int64  
 8   rating        8805 non-null   object 
 9   duration      8806 non-null   object 
 10  listed_in     8809 non-null   object 
 11  description   8809 non-null   object 
 12  Unnamed: 12   0 non-null      float64
 13  Unnamed: 13   0 non-null      float64
 14  Unnamed: 14   0 non-null      float64
 15  Unnamed: 15   0 non-null      float64
 16  Unnamed: 16   0 non-null      float64
 17  Unnamed: 17   0 non-null      float64
 18  Unnamed: 18   0 non-null    

Transformar los datos

In [58]:
# Función para transformar datos

def transform_data(data):
    """
    Realiza la transformación de los datos:
    - Elimina columnas "Unnamed" que no contienen datos válidos.
    - Elimina filas que contienen valores nulos en cualquier columna.

    :param data: DataFrame original con los datos a transformar.
    :return: DataFrame transformado.
    """
    # Eliminar columnas "Unnamed" que están vacías
    unnamed_cols = [col for col in data.columns if 'Unnamed' in col]
    data = data.drop(columns=unnamed_cols)

    # Eliminar filas con cualquier valor nulo en todas las columnas
    data_clean = data.dropna()

    return data_clean

In [59]:
# Uso de la función transform_data
transformed_data = transform_data(data)

# Verificar los datos transformados
transformed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5334 entries, 7 to 8808
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       5334 non-null   object
 1   type          5334 non-null   object
 2   title         5334 non-null   object
 3   director      5334 non-null   object
 4   cast          5334 non-null   object
 5   country       5334 non-null   object
 6   date_added    5334 non-null   object
 7   release_year  5334 non-null   int64 
 8   rating        5334 non-null   object
 9   duration      5334 non-null   object
 10  listed_in     5334 non-null   object
 11  description   5334 non-null   object
dtypes: int64(1), object(11)
memory usage: 541.7+ KB


Cargar los datos en SQL Server

In [60]:
# Función para cargar datos en SQL Server
def load_data(data):
    try:
        # Conectar a SQL Server
        conn = pyodbc.connect(cnxn_str)
        cursor = conn.cursor()

        # Insertar datos
        for index, row in data.iterrows():
            cursor.execute("""
            INSERT INTO   (show_id, title, type, release_year, country, rating, duration, listed_in, description)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, row.show_id, row.title, row.type, row.release_year, row.country, row.rating, row.duration, row.listed_in, row.description)
        
        conn.commit()
        cursor.close()
        conn.close()
        print("Datos cargados correctamente.")
    except Exception as e:
        print('Error al cargar los datos en SQL Server:', e)

# Cargar datos transformados
load_data(transformed_data)


Datos cargados correctamente.
